# Библиотеки

In [1]:
pip install transformers


  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.7/450.7 kB 2.3 MB/s eta 0:00:0000:0100:01
Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (792 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 kB 7.8 MB/s eta 0:00:00:00:01
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install "sglang[all]"


  Using cached setproctitle-1.3.4-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached anthropic-0.42.0-py3-none-any.whl.metadata (23 kB)
  Using cached tiktoken-0.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached vllm-0.6.4.post1-cp38-abi3-manylinux1_x86_64.whl.metadata (10 kB)
  Using cached cuda_python-12.6.2.post1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
INFO: pip is looking at multiple versions of sglang[srt] to determine which version is compatible with other requirements. This could take a while.
  Using cached sglang-0.4.0.post2-py3-none-any.whl.metadata (22 kB)
  Using cached sglang-0.4.0.post1-py3-none-any.whl.metadata (22 kB)
  Using cached sglang-0.4.0-py3-none-any.whl.metadata (22 kB)
INFO: pip is still looking at multiple versions of sglang[srt] to determine which version is compatible with other requirements. This c

In [3]:
pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/


Looking in indexes: https://flashinfer.ai/whl/cu121/torch2.4/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.8/405.8 MB 6.6 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of flashinfer to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.8/405.8 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 3.2 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install 'accelerate>=0.26.0'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 699.3 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


**Шаблон обращения к LLM**

In [ ]:
import openai

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="gemma-2-9b-it",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "Расскажи анекдот"},
    ],
    temperature=0.1,
    max_tokens=500,
)
print(response.choices[0].message.content)


# Qwen2-VL-72B-Instruct-AWQ (только текст (без картинок) работает)

In [5]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/3 итерация/items_1000.jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/3 итерация/data_vector_gt_1000.txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    # print(type(item[1]))
    # print(item[2])
    request = {
        "role": "user",
        "content": f"""
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар и его характеристики поисковому запросу.

Классы запросов:
relevant - товары, которые можно купить по указанному запросу
irrelevant - товары, которые было бы странно увидеть по указанному запросу

Товар:
{item[1]['товар']}

Запрос:
{item[2]['запрос']}

Определи релевантность запроса товару. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.
"""
    }
    
    response = client.chat.completions.create(
        model="Qwen2-VL-72B-Instruct-AWQ ",
        messages=[
            {"role": "system", 
             "content": "You are a helpful AI assistant"},
            request
        ],
        temperature=0,
        max_tokens=3,
    )
    
    prediction = response.choices[0].message.content.strip().lower()
    
    # сравниваю предсказание с правильным ответом, исключая #N/A
    ground_truth_label = ground_truth[i].strip().lower()
    if ground_truth_label != '#n/a':
        is_correct = prediction == ground_truth_label
        results.append({
            "запрос": item[2]['запрос'],
            "товар": item[1]['товар'],
            "предсказание": prediction,
            "реальность": ground_truth_label,
            "is_correct": is_correct
        })

# Подсчет общей Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0

print(f"Accuracy for relevant: {relevant_accuracy:.2%}")
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%}")

Accuracy: 80.67%
Accuracy for relevant: 95.01%
Accuracy for irrelevant: 58.54%


# qwen32b
**Базовый промпт:**

Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар и его характеристики поисковому запросу.

Классы запросов:
relevant - товары, которые можно купить по указанному запросу
irrelevant - товары, которые было бы странно увидеть по указанному запросу

Товар:
{item[1]['товар']}

Запрос:
{item[2]['запрос']}

Определи релевантность запроса товару. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.


# 1000 примеров

In [14]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/3 итерация/items_1000.jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/3 итерация/data_vector_gt_1000.txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    # print(type(item[1]))
    # print(item[2])
    request = {
        "role": "user",
        "content": f"""
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар и его характеристики поисковому запросу.

Классы запросов:
relevant - товары, которые можно купить по указанному запросу
irrelevant - товары, которые было бы странно увидеть по указанному запросу

Товар:
{item[1]['товар']}

Запрос:
{item[2]['запрос']}

Определи релевантность запроса товару. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.
"""
    }
    
    response = client.chat.completions.create(
        model="qwen32b",
        messages=[
            {"role": "system", 
             "content": "You are a helpful AI assistant"},
            request
        ],
        temperature=0,
        max_tokens=3,
    )
    
    prediction = response.choices[0].message.content.strip().lower()
    
    # сравниваю предсказание с правильным ответом, исключая #N/A
    ground_truth_label = ground_truth[i].strip().lower()
    if ground_truth_label != '#n/a':
        is_correct = prediction == ground_truth_label
        results.append({
            "запрос": item[2]['запрос'],
            "товар": item[1]['товар'],
            "предсказание": prediction,
            "реальность": ground_truth_label,
            "is_correct": is_correct
        })

# Подсчет общей Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0

print(f"Accuracy for relevant: {relevant_accuracy:.2%}")
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%}")

Accuracy: 77.71%
Accuracy for relevant: 90.96%
Accuracy for irrelevant: 62.20%


# Другие промпты, чтобы увеличить Accuracy for irrelevant

In [16]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/3 итерация/items_1000.jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/3 итерация/data_vector_gt_1000.txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    # print(type(item[1]))
    # print(item[2])
    request = {
        "role": "user",
        "content": f"""
Ты помощник поиска в маркетплейсе, специализирующийся на разметке негативных пар запрос-товар. Твоя задача — определить, абсолютно ли не соответствует товар поисковому запросу.
**Классы запросов:**
- **irrelevant:** Товар и запрос не имеют абсолютно никакой связи. Нет общих категорий, функциональности, ключевых слов или тематик. Показ такого товара в ответ на запрос будет полностью неожиданным и неуместным для пользователя. Пример: Запрос "кофемашина", а товар — "велосипед".
- **relevant:** Любая связь, даже самая отдаленная или косвенная. Если товар и запрос могут быть хоть как-то связаны по смыслу, категории, использованию или контексту, классифицируй как "relevant". Пример: Запрос "кофемашина", а товар — "кофейные зерна".
**Товар:**
{item[1]['товар']}
**Запрос:**
{item[2]['запрос']}
Отвечай только "relevant" или "irrelevant". Твой ответ должен быть максимально точным, чтобы избежать ложных негативов. Помни, "irrelevant" используется только в случае полного отсутствия связи между товаром и запросом.
"""
    }
    
    response = client.chat.completions.create(
        model="qwen32b",
        messages=[
            {"role": "system", 
             "content": "You are a helpful AI assistant"},
            request
        ],
        temperature=0,
        max_tokens=3,
    )
    
    prediction = response.choices[0].message.content.strip().lower()
    
    # сравниваю предсказание с правильным ответом, исключая #N/A
    ground_truth_label = ground_truth[i].strip().lower()
    if ground_truth_label != '#n/a':
        is_correct = prediction == ground_truth_label
        results.append({
            "запрос": item[2]['запрос'],
            "товар": item[1]['товар'],
            "предсказание": prediction,
            "реальность": ground_truth_label,
            "is_correct": is_correct
        })

# Подсчет общей Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0

print(f"Accuracy for relevant: {relevant_accuracy:.2%}")
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%}")

Accuracy: 80.76%
Accuracy for relevant: 96.67%
Accuracy for irrelevant: 42.68%


# Смотрю ошибки вручную

In [23]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/3 итерация/items_1000.jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/3 итерация/data_vector_gt_1000.txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    # print(type(item[1]))
    # print(item[2])
    request = {
        "role": "user",
        "content": f"""
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар и его характеристики поисковому запросу.

Классы запросов:
relevant - товары, которые можно купить по указанному запросу
irrelevant - товары, которые было бы странно увидеть по указанному запросу

Товар:
{item[1]['товар']}

Запрос:
{item[2]['запрос']}

Определи релевантность запроса товару. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.
"""
    }
    
    response = client.chat.completions.create(
        model="qwen14b",
        messages=[
            {"role": "system", 
             "content": "You are a helpful AI assistant"},
            request
        ],
        temperature=0,
        max_tokens=3,
    )
    
    prediction = response.choices[0].message.content.strip().lower()
    
    # сравниваю предсказание с правильным ответом, исключая #N/A
    ground_truth_label = ground_truth[i].strip().lower()
    if ground_truth_label != '#n/a':
        is_correct = prediction == ground_truth_label
        results.append({
            "запрос": item[2]['запрос'],
            "товар": item[1]['товар'],
            "предсказание": prediction,
            "реальность": ground_truth_label,
            "is_correct": is_correct
        })

In [26]:
for item in results:
    if not item['is_correct'] and item['реальность']=='irrelevant':
        print(item['запрос'])
        print(item['товар'], "\n")

apple vision
Двухкомпонентный Чехол ROCK Vision TPU-PC для Apple iPhone 8 Plus/7 Plus серый 

наволочка дакимакуры подушки арсений попов импровизация
Подушка СувенирShop Комики "Арсений Попов/Импровизация" 35x35 

корзинки вязанные
Плед вязаный SELENA АЛИНА корзинка 220х200 см СЕРЫЙ 

стиральные машины под раковину
Раковина над стиральной машиной Teymi Kati Pro 60х60, литьевой мрамор T50410 

косметика для женшин
Шампунь Kapous Professional С маслами авокадо и оливы 1000 мл 

ремешок mi band
Смарт-часы Mi Smart Band 8 черный CN 

ugreen usb a 3.0 -usb c 3.1 купить
Адаптер uGreen US276 (50533) USB 3.0-A to USB-C M/F Adpater серый 

игрушка для ребенка 1 год
Музыкальная игрушка «Бубен: С Новым годом!» 

рамка qashqai j10
Рамка NISSAN Qashqai 2007-2014, 9" (Incar RNS-FC471) 

флешка микро самсунг
Флешка Samsung BAR Plus 64ГБ Black (MUF-64BE4/APC) 

телевизор белый 32
Рамка для телевизора Samsung 32'' The Frame White (VG-SCFT32WT) 

купить ремещок для часов касио amw 702d
Ремешок для часов

# Добавила характеристики товаров

In [13]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/3 итерация/items_1000.jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/3 итерация/data_vector_gt_1000.txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    # print(type(item[1]))
    # print(item[2])
    request = {
        "role": "user",
        "content": f"""
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар и его характеристики поисковому запросу.
Классы запросов:
- relevant: товары, которые можно купить по указанному запросу
- irrelevant: товары, которые было бы странно увидеть по указанному запросу
Обязательно учитывай, чтобы характеристики товара, указанные в запросе, совпадали с характеристиками товара.
Товар:
{item[1]['товар']}
Характеристика товара:
{item[3]["характеристики"]}
Запрос:
{item[2]['запрос']}
Определи релевантность запроса товару. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.
"""
    }
    
    response = client.chat.completions.create(
        model="qwen32b",
        messages=[
            {"role": "system", 
             "content": "You are a helpful AI assistant"},
            request
        ],
        temperature=0,
        max_tokens=3,
    )
    
    prediction = response.choices[0].message.content.strip().lower()
    
    # сравниваю предсказание с правильным ответом, исключая #N/A
    ground_truth_label = ground_truth[i].strip().lower()
    if ground_truth_label != '#n/a':
        is_correct = prediction == ground_truth_label
        results.append({
            "запрос": item[2]['запрос'],
            "товар": item[1]['товар'],
            "предсказание": prediction,
            "реальность": ground_truth_label,
            "is_correct": is_correct
        })

# Подсчет общей Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0

print(f"Accuracy for relevant: {relevant_accuracy:.2%}")
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%}")

Accuracy: 76.76%
Accuracy for relevant: 88.23%
Accuracy for irrelevant: 78.05%


# gemma-2-9b-it

In [15]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/3 итерация/items_1000.jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/3 итерация/data_vector_gt_1000.txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    # print(type(item[1]))
    # print(item[2])
    request = {
        "role": "user",
        "content": f"""
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар и его характеристики поисковому запросу.
Классы запросов:
relevant - товары, которые можно купить по указанному запросу
irrelevant - товары, которые было бы странно увидеть по указанному запросу
Товар:
{item[1]['товар']}
Запрос:
{item[2]['запрос']}
Определи релевантность запроса товару. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.
"""
    }
    
    response = client.chat.completions.create(
        model="gemma-2-9b-it",
        messages=[
            request
        ],
        temperature=0,
        max_tokens=3,
    )
    
    prediction = response.choices[0].message.content.strip().lower()
    
    # сравниваю предсказание с правильным ответом, исключая #N/A
    ground_truth_label = ground_truth[i].strip().lower()
    if ground_truth_label != '#n/a':
        is_correct = prediction == ground_truth_label
        results.append({
            "запрос": item[2]['запрос'],
            "товар": item[1]['товар'],
            "предсказание": prediction,
            "реальность": ground_truth_label,
            "is_correct": is_correct
        })

# Подсчет общей Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0

print(f"Accuracy for relevant: {relevant_accuracy:.2%}")
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%}")

Accuracy: 79.81%
Accuracy for relevant: 99.17%
Accuracy for irrelevant: 4.88%
